# Summarize Multiple Documents
Uses the recommendations and related paths to focus an LLM summerization on the relevant information.

This notebook depends on the Content nodes already having summaries included in metadata.
This is performed by either the "Abstractive Summary" notebook or the 

The result of this step includes:
- Summary nodes, connected to Content nodes with a SUMMARIZES relationship and to Recommendation nodes with a FOCUS_ON relationship

## Setup

In [ ]:
import os
import logging
import dotenv
from Summarizer import getSummarizer

dotenv.load_dotenv()  # Load environment variables from .env file

## Parameters
OpenTLDR workflows use the notebook block tagged as "parameters" to inject variables (for example to change the LLM model).

> **Do Not Change Variable Names in the Parameters Block** you are welcome to change the values of these parameter variables, but please do not change their names. They are used elsewhere in the notebook and in other workflow processes.

In [ ]:
#Parameters

# If you set the llm_config to None, it will use the environment variable LLM_CONFIG
# Otherwise, here are some options (to run an LLM locally, you will need to download the model to your local machine)
# llm_config = {'type': 'GPT4ALL', 'device':'gpu', 'model':'../LLM_Models/mistral-7b-openorca.gguf2.Q4_0.gguf'}
# llm_config = {'type': 'Ollama', 'model':'mistral:latest'}
# llm_config = {'type': 'ChatGPT', 'model':'gpt-4'}
llm_config = None

llm_prompt = '''
        How do the following articles address: "{request}"?
        In your response, first identify what the articles have in common.
        Second, your response should include highlights of the main differences between the articles.
        The articles below are preceded by their title and separated from the other summaries using '========'. \n
        Summarize the following articles:\n {multicontent}
    '''

# Logging level ranges are (from least to most verbose): ERROR, WARN, INFO, DEBUG
logging_level = logging.INFO

# List of the UniqueIds to Ingest
list_of_uids = None

# level of unnecessary output
verbose = True

In [ ]:
from opentldr import KnowledgeGraph
import opentldr.Domain as domain

logging.getLogger("OpenTLDR").setLevel(logging_level)

kg=KnowledgeGraph()

### Load Content

In [ ]:
if list_of_uids is None:
    list_of_uids = kg.get_all_node_uids_by_tag("Request")

if verbose:
    print ("Found {} Request nodes to attempt multi-document summarization.".format(len(list_of_uids)))

## Run an LLM Model
This notebook uses the `Summarizer` class to run an LLM model. 
You can set the LLM model by changing the `llm_config` variable in the parameters block above or setting LLM_CONFIG in the .env file or environment variable.

In [ ]:
import Summarizer
llm:Summarizer = Summarizer.getSummarizer(llm_config, logging_level=logging_level)

## Build the prompt and run the LLM
This includes the original content, the request it is tailored for, and the explaination of the shortest path through the knowledge graph used to connect them.

In [ ]:
def run_group_summary(kg:KnowledgeGraph, request:domain.Request) -> str:
    len_original=0
    len_summaries=0
    multisum=""
    multidoc_summary="Sorry - No Group Summary Created."
    print("summarizing recommendations for request {}".format(request.title))

    recommendations = kg.get_recommendations_by_request(request)
    # in the case there is only one, just sent that summary instead
    if len(recommendations) == 1:
        return "Only one recommendation."

    for recommendation in recommendations:
        for content in recommendation.recommends:
            print ("Recommended {title} ({score}): {url}".format(title=content.title, score=round(recommendation.score,3),url=content.url))
            len_original += len(content.text)

            if content.metadata is not None:
                summary=content.metadata["summary"]
                len_summaries += len(summary)

                if len(summary) > 0:
                    multisum+="\n\n========n\n\nArticle Title: {}:\n".format(content.title)
                    multisum+="{}\n".format(summary)

            else:
                print("skipping - no metadata summary found")

    if len(multisum) > 0:
        prompt_text = llm_prompt.format(multicontent=multisum, request=request.text)
        multidoc_summary= llm.summarize(prompt_text)
        
        print("\nSummary reduced {reduction}% of content:\t{text}".format(reduction=round(((len_original-len(multidoc_summary))/len_original)*100,1),text=multidoc_summary))

    return multidoc_summary

In [ ]:
for uid in list_of_uids:
    request = kg.get_request_by_uid(uid)

    summary = run_group_summary(kg, request)

    tldr = kg.get_tldr_by_request(request)

    if tldr is not None:
        tldr.metadata["summary"]= summary
        tldr.save()
    else:
        print(summary)
    

In [ ]:
kg.close()